    El archivo homicidios viene con dos hojas de excel que son victimas y hechos, abro cada una en un notebook diferente para modificar y tranformar los valores que se necesite

In [ ]:
pip install fastparquet

In [ ]:
# Se incluye en esta basde de datos cualquier persona muerta ó herida como consecuencia de un siniestro vial

In [109]:
import json
import pandas as pd
import gzip
import ast
from datetime import datetime

In [110]:
# Abro libro hechos del archivo victimas y lo dejo como dataframe de csv

victimas_original = r"C:\Users\Usuario\Henry\PI2_DA\Consigna\victimas_original.csv"

In [111]:
df_victimas = pd.read_csv(victimas_original)

In [ ]:
df_victimas

In [ ]:
df_victimas.info()

In [ ]:
# Verifico con la función la calidad de los datos con la función

In [114]:
def verificar_tipo_datos(df):
# Genero Diccionario vació para almacenar información sobre cada columna
    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}
    
# Iterar sobre  las columnas del DataFrame df
    for columna in df.columns:
        # Calcular el porcentaje de valores no nulos en la columna actual
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        
# Agregar información de mi_dict
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].apply(type).unique())
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100 - porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())
        
# Crear un DataFrame con la información recopilada
    df_info = pd.DataFrame(mi_dict)
    
# Calcular el total de valores nulos en el DataFrame
    total_nulos = df.isnull().sum().sum()
    
# Imprimir el total de valores nulos
    print(f'Total de valores nulos en el DataFrame: {total_nulos}')
    
# Mostrar las columnas con valores nulos y sus respectivos porcentajes (ordenadas de mayor a menor)
    columnas_con_nulos = df_info[df_info["nulos"] > 0][["nombre_campo", "nulos_%"]].sort_values(by="nulos_%", ascending=False)
    if not columnas_con_nulos.empty:
        print("\nColumnas con valores nulos y sus porcentajes:")
        print(columnas_con_nulos)
        
# Devolver el DataFrame con la información de cada columna

    return df_info

In [ ]:
verificar_tipo_datos(df_victimas)

In [ ]:
df_victimas.info()

In [ ]:
# Veo que no hay nulos y esta super. Tengo una columna en comun con con hechos 
# cambiaré el título po ID para poder hacer merge. la fecha se repetiria así que borrare    faltaría sexo, edad, fecha fallecimiento y ROL

In [117]:
# Renombro columna para hacer match
df_victimas = df_victimas.rename(columns={'ID_hecho':'ID'})

In [ ]:
df_victimas.info()

In [119]:
# Borro columnas referentes a fecha

columnas_borrar = ['FECHA', 'AAAA', 'MM', 'DD']
df_victimas = df_victimas.drop(columnas_borrar, axis=1)


In [120]:
df_victimas.head()

,ID,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [121]:
# Grabo el df que esta listo para el análisis

df_victimas.to_parquet('df_victimas.parquet')
